In [83]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [84]:
ls

blender.csv       gbtree.csv            stacked.csv  transactions.csv
burn_blender.csv  PID_Categories.csv    test.csv     Untitled.ipynb
burn_gb_0.csv     SampleSubmission.csv  Train.csv


In [85]:
dataset=pd.read_csv("Train.csv")
dataset

,date,acc,PID
0,2019-12-13 11:38:33.226,256787571627,80G0
1,2019-11-12 11:46:23.755,256789598703,WLIW
2,2019-11-10 06:12:27.728,256753510223,U1DD
3,2019-12-20 13:34:17.453,256702652564,T0LZ
4,2019-11-15 09:16:06.591,256704005298,80G0
...,...,...,...
66509,2020-02-22 21:43:49.110,256706118107,WLIW
66510,2020-02-22 21:54:22.426,256759760017,WLIW
66511,2020-02-22 21:54:46.918,256797135398,DJGS
66512,2020-02-22 22:07:44.173,256000000000,U1DD


In [86]:
dataset_train,dataset_test=train_test_split(dataset,random_state=1)

In [87]:
uganda_public_holidays=[
    (1,1),(26,1),(16,2),
    (8,3),(25,12),(26,12)
]

def is_last_weekend(datetime_obj):
    is_last_week=(datetime_obj.day//7 == 4)
    is_week_end=datetime_obj.weekday() in (5,6)
    return is_last_week and is_week_end

def is_public(datetime_obj):
    day=datetime_obj.day
    month=datetime_obj.month
    return (day,month) in uganda_public_holidays

def is_tommorow_public(datetime_obj):
    tommorow=datetime_obj + datetime.timedelta(days=1)
    return is_public(tommorow)

In [88]:
def transform_dataset(input_dataset):
    # convert the input to purely date format
#     input_dataset['date']=pd.to_datetime(input_dataset['date'].apply(lambda x: x.split(" ")[0]))
    #  get the day 
    input_dataset['day']=input_dataset['date'].dt.day
    # get the month
    input_dataset['month']=input_dataset['date'].dt.month
    #day of week
    input_dataset['day_of_week']=input_dataset['date'].dt.weekday
    # get the week of month
    input_dataset['week_of_month']=input_dataset['date'].apply(lambda x:x.day//7)
    # get the quarter of year
    input_dataset['quarter_of_year']=input_dataset['date'].dt.month//4
    #is the day a public holiday
    input_dataset['public_holiday']=input_dataset['date'].map(is_public)
    #is the day after a public holiday
    input_dataset['tommorow_public_holiday']=input_dataset['date'].map(is_tommorow_public)
    #is today a weekend
    input_dataset['is_week_end']=input_dataset['day_of_week'].map(lambda x:1 if x in [5,6] else 0)
    #is today the last weekend
    input_dataset['is_last_week_end']=input_dataset['date'].apply(is_last_weekend)
    
    #create a target variable
    return input_dataset

In [89]:
# generate the negative class for the dataset

In [90]:
# unique_transactions
unique_transactions=pd.unique(dataset['PID'])

In [91]:
dataset.sort_values(by=['date',"acc"]).head(10)

,date,acc,PID
929,2019-11-01 00:38:59.418,256702177173,U1DD
17858,2019-11-01 00:41:40.281,256702177173,80G0
30451,2019-11-01 00:58:35.659,256774747834,U1DD
17853,2019-11-01 01:00:57.025,256774747834,80G0
4005,2019-11-01 01:02:30.683,256703053300,DJGS
23388,2019-11-01 01:24:14.839,256756309678,WLIW
32826,2019-11-01 01:42:57.937,256772001846,U1DD
7980,2019-11-01 01:45:28.100,256756309678,T0LZ
24658,2019-11-01 01:46:57.332,256772001846,80G0
19935,2019-11-01 02:04:35.010,256772541327,WLIW


In [117]:
dataset['key']=dataset['date'].astype(str)+"|"+dataset['acc'].astype(str)

In [118]:
trxns=dataset.groupby('key')['PID'].unique()

In [227]:
trxns

key
2019-11-01|256000000000    [Q9SJ, U1DD, 6Q4Z, 3USE]
2019-11-01|256312290313                [CPEH, 80G0]
2019-11-01|256393100610          [6Q4Z, 80G0, DJGS]
2019-11-01|256700117509                      [WLIW]
2019-11-01|256700126270                [U1DD, 80G0]
                                     ...           
2020-02-22|256791696010                      [CPEH]
2020-02-22|256793022988                      [80G0]
2020-02-22|256793241371                      [WLIW]
2020-02-22|256794953575                      [80G0]
2020-02-22|256797135398                      [DJGS]
Name: PID, Length: 23803, dtype: object

In [151]:
negative_class=[]
for index,transactions in zip(trxns.index,trxns):
    date,acc=index.split("|")
    for unique_trxn in unique_transactions:
        if unique_trxn not in transactions:
            negative_class.append([
                    date,
                    acc,
                    unique_trxn,
                    0    
                ])

In [176]:
negative_class_df=pd.DataFrame(negative_class,columns=['date','acc','PID','target'])
negative_class_df['date']=pd.to_datetime(negative_class_df['date'])

In [166]:
input_dataset=dataset[['date','acc','PID']]

In [167]:
input_dataset['target']=input_dataset['PID'].map(lambda x: 1)

In [219]:
full_dataset=pd.concat([input_dataset,negative_class_df])
full_dataset['acc']=full_dataset['acc'].astype(str)

In [220]:
full_dataset

,date,acc,PID,target
0,2019-12-13,256787571627,80G0,1
1,2019-11-12,256789598703,WLIW,1
2,2019-11-10,256753510223,U1DD,1
3,2019-12-20,256702652564,T0LZ,1
4,2019-11-15,256704005298,80G0,1
...,...,...,...,...
774243,2020-02-22,256797135398,AWYV,0
774244,2020-02-22,256797135398,NFDD,0
774245,2020-02-22,256797135398,B5DH,0
774246,2020-02-22,256797135398,XGSZ,0


In [221]:
final_train=transform_dataset(full_dataset)

In [222]:
final_train

,date,acc,PID,target,day,month,day_of_week,week_of_month,quarter_of_year,public_holiday,tommorow_public_holiday,is_week_end,is_last_week_end
0,2019-12-13,256787571627,80G0,1,13,12,4,1,3,False,False,0,False
1,2019-11-12,256789598703,WLIW,1,12,11,1,1,2,False,False,0,False
2,2019-11-10,256753510223,U1DD,1,10,11,6,1,2,False,False,1,False
3,2019-12-20,256702652564,T0LZ,1,20,12,4,2,3,False,False,0,False
4,2019-11-15,256704005298,80G0,1,15,11,4,2,2,False,False,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
774243,2020-02-22,256797135398,AWYV,0,22,2,5,3,0,False,False,1,False
774244,2020-02-22,256797135398,NFDD,0,22,2,5,3,0,False,False,1,False
774245,2020-02-22,256797135398,B5DH,0,22,2,5,3,0,False,False,1,False
774246,2020-02-22,256797135398,XGSZ,0,22,2,5,3,0,False,False,1,False


In [255]:
final_train.to_csv("transactions.csv",index=False)

In [223]:
final_train['target'][final_train['target']==0]

0         0
1         0
2         0
3         0
4         0
         ..
774243    0
774244    0
774245    0
774246    0
774247    0
Name: target, Length: 774248, dtype: int64

In [1]:
ls

blender.csv       PID_Categories.csv    Train.csv
burn_blender.csv  SampleSubmission.csv  transactions.csv
gbtree.csv        test.csv              Untitled.ipynb


In [49]:
import pandas as pd

In [52]:
results=pd.read_csv('blender.csv')

In [53]:
blender_predictions=results['Prediction']

In [51]:
gb_predictions=results['Prediction']

In [31]:
results['Prediction']=results['Prediction'].map(lambda x:x if x>0 else 0)

In [54]:
output=pd.read_csv("test.csv")[['ID']]

In [56]:
avg=blender_predictions+gb_predictions

In [58]:
avg.values

array([ 0.21336514,  0.20843619,  0.31669599, ..., -0.00071267,
        0.00235546, -0.00236639])

In [61]:
output['burn_area']=avg.values

In [70]:
output.to_csv("stacked.csv",index=False)

In [64]:
output['burn_area']=output['burn_area']/2

In [66]:
output['burn_area']=output['burn_area'].apply(lambda x:x if x>0 else 0)

In [67]:
output

,ID,burn_area
0,0_2014-01-01,0.106683
1,1_2014-01-01,0.104218
2,2_2014-01-01,0.158348
3,3_2014-01-01,0.151757
4,4_2014-01-01,0.169243
...,...,...
137551,3816_2016-12-01,0.001866
137552,3817_2016-12-01,0.002005
137553,3818_2016-12-01,0.000000
137554,3819_2016-12-01,0.001178


In [71]:
ls

blender.csv       gbtree.csv            stacked.csv  transactions.csv
burn_blender.csv  PID_Categories.csv    test.csv     Untitled.ipynb
burn_gb_0.csv     SampleSubmission.csv  Train.csv


In [73]:
submission=pd.read_csv('SampleSubmission.csv')

In [79]:
submission['date']=pd.to_datetime(submission['Account X date X PID'].map(lambda x:x.split("X")[1]))

In [81]:
submission.drop("Prediction",1,inplace=True)

In [82]:
submission

,Account X date X PID,date
0,256786974320 X 2020-02-23 X 5YR6,2020-02-23
1,256786974320 X 2020-02-23 X WLIW,2020-02-23
2,256786974320 X 2020-02-23 X U1DD,2020-02-23
3,256786974320 X 2020-02-23 X T0LZ,2020-02-23
4,256786974320 X 2020-02-23 X 80G0,2020-02-23
...,...,...
111043,256704700785 X 2020-02-29 X P4IQ,2020-02-29
111044,256704700785 X 2020-02-29 X PNYC,2020-02-29
111045,256704700785 X 2020-02-29 X YDAG,2020-02-29
111046,256704700785 X 2020-02-29 X ZDTG,2020-02-29


In [93]:
import datetime

In [95]:
test=transform_dataset(submission)

In [110]:
test.to_csv("number2.csv",index=False)

In [109]:
test['PID']=test['Account X date X PID'].map(lambda x:x.split(" X ")[-1].strip())
test['acc']=test['Account X date X PID'].map(lambda x:x.split(" X ")[0].strip())

In [111]:
test

,Account X date X PID,date,day,month,day_of_week,week_of_month,quarter_of_year,public_holiday,tommorow_public_holiday,is_week_end,is_last_week_end,PID,acc
0,256786974320 X 2020-02-23 X 5YR6,2020-02-23,23,2,6,3,0,False,False,1,False,5YR6,256786974320
1,256786974320 X 2020-02-23 X WLIW,2020-02-23,23,2,6,3,0,False,False,1,False,WLIW,256786974320
2,256786974320 X 2020-02-23 X U1DD,2020-02-23,23,2,6,3,0,False,False,1,False,U1DD,256786974320
3,256786974320 X 2020-02-23 X T0LZ,2020-02-23,23,2,6,3,0,False,False,1,False,T0LZ,256786974320
4,256786974320 X 2020-02-23 X 80G0,2020-02-23,23,2,6,3,0,False,False,1,False,80G0,256786974320
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111043,256704700785 X 2020-02-29 X P4IQ,2020-02-29,29,2,5,4,0,False,False,1,True,P4IQ,256704700785
111044,256704700785 X 2020-02-29 X PNYC,2020-02-29,29,2,5,4,0,False,False,1,True,PNYC,256704700785
111045,256704700785 X 2020-02-29 X YDAG,2020-02-29,29,2,5,4,0,False,False,1,True,YDAG,256704700785
111046,256704700785 X 2020-02-29 X ZDTG,2020-02-29,29,2,5,4,0,False,False,1,True,ZDTG,256704700785


In [133]:
gb=pd.read_csv("t_gb.csv")

In [130]:
test['Prediction']=blender['PredictedLabel']

In [168]:
test[['Account X date X PID','Prediction']].to_csv("2_simple_stack_2.5.csv",index=False)

In [127]:
ls

2_blender.csv     burn_gb_0.csv       SampleSubmission.csv  t_gb.csv
2_gb.csv          gbtree.csv          stacked.csv           Train.csv
blender.csv       number2.csv         t_blender.csv         transactions.csv
burn_blender.csv  PID_Categories.csv  test.csv              Untitled.ipynb


In [128]:
test['Prediction'].dtype

dtype('int64')

In [138]:
total=gb['Prediction']+blender['Prediction']

In [163]:
average=total/2

In [164]:
test['Prediction']=average

In [166]:
test['Prediction']

0         0.043047
1         0.412276
2         0.258753
3         0.128244
4         0.357570
            ...   
111043    0.001014
111044    0.000732
111045    0.000595
111046    0.001071
111047    0.008898
Name: Prediction, Length: 111048, dtype: float64

In [167]:
test['Prediction']=test['Prediction'].apply(lambda x:1 if x>0.60 else 0)